# Ingesting Prochaska+11 CGM dataset [v1.0]

In [25]:
# imports
from astropy.table import Table
from astropy.coordinates import SkyCoord

import pyigm
from pyigm.cgm.cgmsurvey import CGMAbsSurvey

## Summary

In [17]:
ovi_file = pyigm.__path__[0]+'/data/CGM/P11/lowovidat.fits'

In [18]:
ovidat = Table.read(ovi_file)

In [23]:
ovidat[0:3]

QSO,QSO_RA,QSO_DEC,QSO_ZEM,QSO_VMAG,QSO_UV,FLG_GAL,GAL_FIL,R_LIMIT,N_GAL [4],"COMPLETE [2,4]",FLG_FUSE,FUSE_EXP,FUSE_SNR,FLG_STIS,STIS_COMM [4],FLG_GHRS,GHRS_COMM [4]
str15,str15,str15,float32,float32,float32,int16,str58,float32,int32,int32,int16,float32,float32,int16,str60,int16,str60
Q0026+1259,00:29:13.8,+13:16:04,0.142,14.78,20.0,1,/u/xavier/LCO/OVI/FUSE/data/Q0026+1259/Q0026+1259_gal.fits,20.0,131 .. 47,22 .. 85,1,20000.0,7.0,0,..,1,G270M 5222s ..
TONS180,00:57:20.0,-22:22:56,0.06198,16.6,30.0,1,/u/xavier/LCO/OVI/FUSE/data/TONS180/TONS180_gal.fits,19.7,110 .. 4,15 .. 92,1,132453.0,15.0,2,G140M 7000s 15. ..,0,..
TONS210,01:21:51.5,-28:20:57,0.116,14.7,70.0,1,/u/xavier/LCO/OVI/FUSE/data/TONS210/TONS210_gal.fits,20.0,71 .. 5,6 .. 87,1,56500.0,20.0,2,E140M 22000s ..,0,..


In [21]:
qso_radec = SkyCoord(ra=ovidat['QSO_RA'], dec=ovidat['QSO_DEC'], unit=(u.hourangle, u.deg))

## Dwarfs

In [9]:
cgm_dwarf_file = pyigm.__path__[0]+'/data/CGM/P11/dwarf_galabs_strct.fits'

In [10]:
cgm_dwarfs = Table.read(cgm_dwarf_file)

In [11]:
cgm_dwarfs[0:3]

FIELD,ID,OBJ_ID,FLG_ANLY,FLG_SURVEY,OBJ_TYPE,MAG [10],MAGERR [10],FILTER [10],IMG_FIL [10],XYPIX [2],RA,DEC,AREA,STARGAL,GAL_TYPE,GAL_COEFF [10],Z,VCIRC,FSPEC_FIL [10],DRA,DDEC
str11,int32,str1,int16,int16,int16,float32,float32,str1,str27,float32,float64,float64,float32,float32,str5,float32,float64,float32,str29,float64,float64
Q0026+1259,1303,a,7,1,0,19.733 .. 0.0,0.125 .. 9.99,B ..,Images/Q0026+1259XB.fits ..,1014.76 .. 1034.29,7.28886499155,13.2746021629,10.43,0.18,Late,0.331349 .. 0.0,0.0329451337457,0.0,..,45.6990947951,0.0191390593786
TONS180,2295,a,7,1,0,18.923 .. 1.0,0.088 .. 0.05,B ..,Images/TONS180XB.fits ..,1318.89 .. 607.18,14.2667432785,-22.44755991,10.92,0.19,Late,-0.0115093 .. 0.0,0.0233643911779,0.0,..,154.07390626,0.0207054292147
PKS0405-123,90033,a,7,1,0,0.0 .. 0.44,0.0 .. 0.06,..,..,61.9512 .. -12.1839,61.9512481689,-12.1838884354,0.0,0.0,Late,0.0 .. 0.0,0.167,1.0,..,97.7760719383,0.0877822129587


### Funny columns -- Renaming

In [12]:
cgm_dwarfs.rename_column('DRA', 'rho(kpc)')
cgm_dwarfs.rename_column('DDEC', 'L(L*)')

#### MAG has HI
    2 == Ref
    3 == z
    4 == Lya EW
    5 == sigEW Lya
    8 == NHI
    9 == sigNHI

In [14]:
cgm_dwarfs[1]['MAG']

array([  1.89230003e+01,   1.80701466e+01,   5.00000000e+00,
         2.34109219e-02,   2.22000000e+02,   2.90000000e+01,
         5.60000000e+01,   4.00000000e+00,   1.38000002e+01,
         1.00000000e+00], dtype=float32)

#### MAGERR has OVI
    2 == Ref
    3 == z
    4 == EW 1031
    5 == sigEW 1031
    8 == NOVI    
    9 == sigNOVI

In [15]:
cgm_dwarfs[1]['MAGERR']

array([  8.79999995e-02,   6.49999976e-02,   4.00000000e+00,
         2.33999994e-02,   4.30000000e+01,   1.50000000e+01,
         3.00000000e+01,   0.00000000e+00,   1.34799995e+01,
         5.00000007e-02], dtype=float32)

#### Refs

In [16]:
refdict = {1: 'tripp08', 2: 'tc08a', 3: 'ds08', 4: 'dsr+06', 5: 'pss04', 6: 'cm09', 9: 'p+11'}

## Ingest
    python ingest_lit.py

## Read

In [26]:
p11_tarfile = pyigm.__path__[0]+'/data/CGM/P11/P11_sys.tar'

In [28]:
p11 = CGMAbsSurvey.from_tarball(p11_tarfile, chk_lowz=False)

Skipping a likely folder: CGM_JSON


/Users/xavier/local/Python/linetools/linetools/isgm/abssystem.py:288: UserWarning: Input AbsComponent with Zion=(1, 1) does not match AbsSystem rules. Not appending
  warnings.warn('Input AbsComponent with Zion={} does not match AbsSystem rules. Not appending'.format(abscomp.Zion))


In [29]:
p11

<CGM_Survey:  nsys=54, ref=>
<IGMSystem: IGMSystem 00:29:13.8 13:16:04, 0.0329451, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 00:29:13.8 13:16:04, 0.039311, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 00:57:20 -22:22:56, 0.0233644, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 00:57:20 -22:22:56, 0.045619, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 03:11:55.2 -76:51:51, 0.202643, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 03:11:55.2 -76:51:51, 0.0593531, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.0964516, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.297609, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.352, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.153212, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.203022, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.36124, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.167, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 04:07:48.4 -12:11:37, 0.167043, NHI=0, Z/H=0>
<IGMSystem: IGMSystem 10:07: